# Bayes'sches-Netz zur Klassifikation von Wohnungsbewohnerkategorien (Datensatz "wohnungen_1.csv")
Dieses Notebook stellt ein Modell auf Basis eines Bayes'schen-Netzes vor, das verwendet wird, um die Bewohnerkategorie einer Wohnung basierend auf den Merkmalen der Wohnung zu klassifizieren.

## Ziel
Das Ziel dieses Projekts ist es, die Zusammenhänge zwischen den Eigenschaften einer Wohnung und den Bewohnerkategorien zu modellieren. Wir verwenden ein Bayes'sches-Netz, um Wahrscheinlichkeiten vorherzusagen, auch wenn einige Informationen unvollständig sind.

---

## Vorbereitung
Zuerst werden die benötigten Bibliotheken importiert.

In [2]:
import pandas as pd
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator
from pgmpy.inference import VariableElimination
from data_cleaning.text_to_numeric import get_zimmer, get_stockwerk, get_heizung, get_kindergarten, get_schule, get_bahn, get_miete, get_nebenkosten, get_alter, get_lage, get_kaution, get_kueche, get_bad, get_mobliert, get_quadratmeter

### Erläuterung:
- **Bewohnerkategorie**: Der Datensatz enthält mehrere Spalten, mit verschiedenen Bewohnergruppen, wie DINK (Double Income No Kids) aber auch Alleinerziehend und Studierende, die die Zugehörigkeit zu einer Kategorie angeben. Diese werden in einer neuen Spalte `Bewohnerkategorie` zusammengefasst.
- **Datenbereinigung**: Um den Datensatz korrekt nutzen zu können, müssen wir den Fließtext durch Zahlen ersetzen, damit die KI mit diesen Daten arbeiten kann. Dafür werden "nein" und "ja" in numerische Werte (0 und 1) umgewandelt. Werte wo es mehr als ein Wert gibt wie bei der Miete, wurde die Mitte der beiden Werte genommen. Dies Funktioniert über unser Python Skript "data_cleaning.py".


In [5]:
file_path = '../../data/Wohnungen_1.csv'
data = pd.read_csv(file_path, sep=';') # Gegebenen Datensatz einlesen

# Bewohnerkategorie bestimmen
bewohnerkategorie = ['Studierende', 'Kleinfamilie', 'DINK', 'Alleinerziehende', 'Expatriate', 'Rentnerpaar']
data['Bewohnerkategorie'] = data[bewohnerkategorie].apply(lambda row: 'Keine' if all(row == 'nein') else ', '.join(row[row == 'ja'].index), axis=1)

# Löschen von Daten, die nicht kategorisiert sind (dazu später mehr)
data = data[data['Bewohnerkategorie'] != 'Keine']

# Spalten, in denen "nein" durch 0 und "ja" durch 1 ersetzt werden sollen
columns_to_replace = ["Hausmeister", "Garage", "Aufzug", "Balkon", "Terrasse", "Kehrwoche"]

# Ersetze "nein" und "ja" in den angegebenen Spalten
pd.set_option('future.no_silent_downcasting', True)
data[columns_to_replace] = data[columns_to_replace].replace({'nein': 0, 'ja': 1}).infer_objects(copy=False)

# Fließtext durch Ganzzahl ersetzen
data['Zimmerzahl'] = data['Zimmerzahl'].apply(get_zimmer)
data['Stockwerk'] = data['Stockwerk'].apply(get_stockwerk)
data['Heizung'] = data['Heizung'].apply(get_heizung)
data['Kindergarten'] = data['Kindergarten'].apply(get_kindergarten)
data['Schule'] = data['Schule'].apply(get_schule)
data['S-Bahn'] = data['S-Bahn'].apply(get_bahn)
data['Miete'] = data['Miete'].apply(get_miete)
data['Nebenkosten'] = data['Nebenkosten'].apply(get_nebenkosten)
data['Alter'] = data['Alter'].apply(get_alter)
data['Lage'] = data['Lage'].apply(get_lage)
data['Kaution'] = data['Kaution'].apply(get_kaution)
data['Kueche'] = data['Kueche'].apply(get_kueche)
data['Bad'] = data['Bad'].apply(get_bad)
data['Moebliert'] = data['Moebliert'].apply(get_mobliert)
data['Quadratmeter'] = data['Quadratmeter'].apply(get_quadratmeter)

print(data.head())

    Zimmerzahl  Stockwerk  Heizung  Hausmeister  Kindergarten  Schule  S-Bahn  \
11           2          0        0            0             1       2       4   
13           2          0        5            0             2       2       3   
23           2          1        4            1             1       2       4   
27           0          0        1            0             1       0       0   
40           0          8        5            0             0       2       2   

    Garage  Miete  Nebenkosten  ...  Kehrwoche  Moebliert  Quadratmeter  \
11       0    726          276  ...          0          0           116   
13       1    726          226  ...          0          0           106   
23       0    776          276  ...          0          0           116   
27       0    226           50  ...          0          0            25   
40       0    226           50  ...          0          0            36   

    Studierende  Kleinfamilie  DINK  Alleinerziehende  Expatri

### Modelle erstellen und trainieren
Das Bayes'sche Netz wurde anhand der vorhandenen Daten trainiert, um die Beziehungen zwischen den Merkmalen und der Bewohnerkategorie zu modellieren. Aufgrund der hohen Anzahl von Merkmalen und Kategorien wurde bewusst ein einfacheres Bayes-Netz gewählt, da ein komplexeres Netzwerk rechnerisch aufwendiger und schwerer zu handhaben wäre.

Für das Training des Modells kam der Maximum-Likelihood-Schätzer (ML-Schätzer) zum Einsatz. Dieser wurde gewählt, weil wir annehmen, dass die Daten unabhängig und identisch verteilt sind. Im Gegensatz dazu weist der Bayes-Schätzer eine Tendenz zugunsten der Klasse mit den meisten Beispielen auf. Da die Klassen in unserem Datensatz ungleichmäßig verteilt sind, war der Maximum-Likelihood-Schätzer die geeignetere Wahl.

In [6]:
# Modell 1
model1 = BayesianNetwork([
    ('Zimmerzahl', 'Bewohnerkategorie'),
    ('Stockwerk', 'Bewohnerkategorie'),
    ('Heizung', 'Bewohnerkategorie'),
    ('S-Bahn', 'Bewohnerkategorie'),
    ('Miete', 'Bewohnerkategorie'),
    ('Alter', 'Bewohnerkategorie'),
    ('Lage', 'Bewohnerkategorie')
])
# Modell fitten
model1.fit(data, estimator=MaximumLikelihoodEstimator)
# Inferenz durchführen
inference = VariableElimination(model1)

### Erstes Beispiel - Kleine Änderung
Wir vergleichen die Ergebnisse des Modells anhand der Beispiele "example_Zimmer_difference" und "example_Zimmer_difference2". Dabei variiert nur die Anzahl der Zimmer, um zu sehen, wie sich die Bewohnerkategorie änderen kann.

In [15]:
example_Zimmer_difference = {
    "Zimmerzahl": get_zimmer("Zwei Zimmer"),
    "Stockwerk": get_stockwerk("EG"),
    "Heizung": get_heizung("Erdwaerme"),
    "S-Bahn": get_bahn("erreichbar"),
    "Miete": get_miete("275-277"),
    "Alter": get_alter("62-64 Jahre"),
    "Lage": get_lage("Hauptstrasse")
}

# Wahrscheinlichkeiten für jede Zielklasse berechnen
prediction = inference.query(variables=["Bewohnerkategorie"], evidence=example_Zimmer_difference)

# Wahrscheinlichkeiten in ein Dictionary umwandeln
result = {str(state): float(prob) for state, prob in zip(prediction.state_names["Bewohnerkategorie"], prediction.values)}

# Ausgabe schön formatiert untereinander
print("Wahrscheinlichkeiten für jede Bewohnerkategorie:")
for category, probability in result.items():
    print(f"{category}: {probability:%}")

Wahrscheinlichkeiten für jede Bewohnerkategorie:
DINK: 0.000000%
Expatriate: 0.000000%
Kleinfamilie: 0.000000%
Kleinfamilie, DINK: 0.000000%
Studierende: 0.000000%
Studierende, Alleinerziehende, Rentnerpaar: 100.000000%


In [16]:
example_Zimmer_difference2 = {
    "Zimmerzahl": 2,
    "Stockwerk": 0,
    "Heizung": 0,
    "S-Bahn": 3,
    "Miete": 276,
    "Alter": 63,
    "Lage": 4
}

prediction = inference.query(variables=["Bewohnerkategorie"], evidence=example_Zimmer_difference2)

result = {str(state): float(prob) for state, prob in zip(prediction.state_names["Bewohnerkategorie"], prediction.values)}

print("Wahrscheinlichkeiten für jede Bewohnerkategorie:")
for category, probability in result.items():
    print(f"{category}: {probability:%}")

Wahrscheinlichkeiten für jede Bewohnerkategorie:
DINK: 16.666667%
Expatriate: 16.666667%
Kleinfamilie: 16.666667%
Kleinfamilie, DINK: 16.666667%
Studierende: 16.666667%
Studierende, Alleinerziehende, Rentnerpaar: 16.666667%


### Auswertung der Ergebnisse
Wie oben schon beschrieben, wurde nur die Anzahl der Zimmer variiert um zu sehen, wie sich die Bewohnerkategorie änderen kann. Hierbei wurde festgestellt, dass selbst eine kleine Änderung in einem Merkmal die Wahrscheinlichkeiten für die Bewohnerkategorien beeinflussen kann.

### Potenzielle Erklärung
Die Unterschiede in den Wahrscheinlichkeiten zwischen den beiden Beispielen können auf die Änderung der Zimmerzahl zurückgeführt werden. Eine höhere Zimmerzahl könnte bestimmte Bewohnerkategorien wie "Kleinfamilien" oder "DINK" bevorzugen, während eine niedrigere Zimmerzahl eher für "Studierende" oder "Alleinerziehende" geeignet sein könnte. Diese Ergebnisse zeigen, wie das Bayes'sche Netz die Beziehungen zwischen den Merkmalen der Wohnung und den Bewohnerkategorien modelliert und wie sich Änderungen in einem Merkmal auf die Wahrscheinlichkeiten der Kategorien auswirken können.

In [17]:
#Modell 2
model2 = BayesianNetwork([
    ('Stockwerk', 'Bewohnerkategorie'),
    ('Kindergarten', 'Bewohnerkategorie'),
    ('Schule', 'Bewohnerkategorie'),
    ('Miete', 'Bewohnerkategorie'),
    ('Nebenkosten', 'Bewohnerkategorie'),
    ('Kaution', 'Bewohnerkategorie'),
    ('Kueche', 'Bewohnerkategorie'),
])

model2.fit(data, estimator=MaximumLikelihoodEstimator)
inference = VariableElimination(model2)

### Zweites Beispiel - Viel Änderung
An diesem Modell vergleichen wir die Beispiele 'example_large_changes' und 'example_large_changes2', hierbei varriert sich, im Gegensatz zum Ersten Modell jede Kategorie mit drastischen Änderungen.


In [18]:
example_large_changes = {
    "Stockwerk": get_stockwerk("2.Stock"),
    "Schule": get_schule("nah"),
    "Miete": get_miete("201-250"),
    "Nebenkosten": get_nebenkosten("unter 50"),
    "Kaution": get_kaution("ueber 3000"),
    "Kueche": get_kueche("Kueche (neu)"),
}

prediction = inference.query(variables=["Bewohnerkategorie"], evidence=example_large_changes)

result = {str(state): float(prob) for state, prob in zip(prediction.state_names["Bewohnerkategorie"], prediction.values)}

print("Wahrscheinlichkeiten für jede Bewohnerkategorie:")
for category, probability in result.items():
    print(f"{category}: {probability:.2%}")

Wahrscheinlichkeiten für jede Bewohnerkategorie:
DINK: 16.67%
Expatriate: 16.67%
Kleinfamilie: 16.67%
Kleinfamilie, DINK: 16.67%
Studierende: 16.67%
Studierende, Alleinerziehende, Rentnerpaar: 16.67%


In [19]:
example_large_changes2 = {
    "Stockwerk": get_stockwerk("7.Stock"),
    "Schule": get_schule("fern"),
    "Miete": get_miete("551-600"),
    "Nebenkosten": get_nebenkosten("151-200"),
    "Kaution": get_kaution("keine"),
    "Kueche": get_kueche("Kueche (alt)"),
}

prediction = inference.query(variables=["Bewohnerkategorie"], evidence=example_large_changes2)

result = {str(state): float(prob) for state, prob in zip(prediction.state_names["Bewohnerkategorie"], prediction.values)}

print("Wahrscheinlichkeiten für jede Bewohnerkategorie:")
for category, probability in result.items():
    print(f"{category}: {probability:.2%}")

Wahrscheinlichkeiten für jede Bewohnerkategorie:
DINK: 16.67%
Expatriate: 16.67%
Kleinfamilie: 16.67%
Kleinfamilie, DINK: 16.67%
Studierende: 16.67%
Studierende, Alleinerziehende, Rentnerpaar: 16.67%


### Auswertung der Ergebnisse
Wie oben schon beschrieben, wurden alle im Modell angegeben Wohnungskategorien geändert. Hierbei wurde festgestellt, dass selbst bei größeren Änderungen, die Wahrscheinlichkeiten gleich bleiben kann.

### Potenzielle Erklärung
Eine mögliche Erklärung worauf das zu führen hat, kann sein das die Änderungen in den Merkmalen nicht ausreichen um die Wahrscheinlichkeiten zu beeinflussen. Welches zurückzuführen ist, dass die Merkmale nicht stark genug miteinander korrelieren, um die Bewohnerkategorien signifikant zu beeinflussen. Was davon kommen kann, dass die Beziehungen zwischen den Merkmalen und den Bewohnerkategorien im Modell möglicherweise nicht stark genug sind, um signifikante Änderungen in den Wahrscheinlichkeiten zu bewirken.